## Working with Different Types of Data


### Step 1: Initialize PySpark Session


In [114]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, column, avg, instr, coalesce, regexp_replace, explode, create_map

# Create a Spark session
spark = SparkSession.builder.appName("day3").getOrCreate()


### Step 2: Load the Dataset


In [152]:
# Load the Chipotle dataset into a Spark DataFrame
data_path = "./titanic.csv"  # Replace with the actual path
titanic_df = spark.read.csv(data_path, header=True, inferSchema=True)

# Load the Chipotle dataset into a Spark DataFrame
data_path = './chipotle.csv' # Replace with the actual path
chipotle_df = spark.read.csv(data_path, header=True, inferSchema=True)

# Load the Chipotle dataset into a Spark DataFrame
data_path = './kalimati_tarkari_dataset.csv' # Replace with the actual path
kalimati_df = spark.read.csv(data_path, header=True, inferSchema=True)


In [153]:
print(titanic_df.printSchema(),chipotle_df.printSchema(),kalimati_df.printSchema())

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)

root
 |-- _c0: integer (nullable = true)
 |-- order_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- item_name: string (nullable = true)
 |-- choice_description: string (nullable = true)
 |-- item_price: string (nullable = true)

root
 |-- SN: integer (nullable = true)
 |-- Commodity: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Unit: string (nullable = true)
 |-- Minimum: double (nullable = true)
 |-- Maximum: double (nullable = true)
 |-- Average: double (nullable = true)

None None N

### Converting to Spark Types:

Question: Load the "titanic" dataset and convert the "Fare" column from double to integer.




In [135]:
titanic_df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [154]:
titanic_df_contype = titanic_df.withColumn("Fare", col("Fare").cast("int"))
titanic_df_contype.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+----+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+----+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|  71|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|   7| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|  53| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|          330877|   8| null|  

In [155]:
titanic_df_contype.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: integer (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



### Working with Booleans:

Question: Load the "titanic" dataset and add a new column "IsAdult" that indicates whether a passenger is an adult (age >= 18) or not.

In [156]:
titanic_df_isadult = titanic_df.withColumn("IsAdult", when(col("Age") >= 18, True).otherwise(False))
titanic_df_isadult.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|IsAdult|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|   true|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|   true|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|   true|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|   true|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|   true|
|          6|   

In [21]:
titanic_df_isadultornot = titanic_df.withColumn(
    "IsAdult",
        when(col("age") >= 18, "Adult")
        .otherwise("Juvenile")
)
titanic_df_isadultornot.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+----+-----+--------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|Fare|Cabin|Embarked| IsAdult|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+----+-----+--------+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7| null|       S|   Adult|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|  71|  C85|       C|   Adult|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|   7| null|       S|   Adult|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|  53| C123|       S|   Adult|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8| null|       S|   Adult|
|          6|       0|     3|   

In [20]:
titanic_df.drop("IsAdult").columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

### Working with Numbers:

Question: Load the "titanic" dataset and calculate the average age of male and female passengers separately.

In [157]:
average_age_by_gender = titanic_df.groupBy("Sex").agg(avg(col("Age")).alias("AverageAge"))
average_age_by_gender.show()

+------+------------------+
|   Sex|        AverageAge|
+------+------------------+
|female|27.915708812260537|
|  male| 30.72664459161148|
+------+------------------+



In [27]:
average_age_by_gender = titanic_df.selectExpr("Sex", "avg(Age) as AverageAge").groupBy("Sex").agg({"AverageAge": "first"})
average_age_by_gender.show()

AnalysisException: [MISSING_GROUP_BY] The query does not include a GROUP BY clause. Add GROUP BY or turn it into the window functions using OVER clauses.;
Aggregate [Sex#21, avg(Age#22) AS AverageAge#493]
+- Project [PassengerId#17, Survived#18, Pclass#19, Name#20, Sex#21, Age#22, SibSp#23, Parch#24, Ticket#25, cast(Fare#101 as int) AS Fare#115, Cabin#27, Embarked#28]
   +- Project [PassengerId#17, Survived#18, Pclass#19, Name#20, Sex#21, Age#22, SibSp#23, Parch#24, Ticket#25, cast(Fare#26 as int) AS Fare#101, Cabin#27, Embarked#28]
      +- Relation [PassengerId#17,Survived#18,Pclass#19,Name#20,Sex#21,Age#22,SibSp#23,Parch#24,Ticket#25,Fare#26,Cabin#27,Embarked#28] csv


### Working with Strings:

Question: Load the "chipotle" dataset and find the item names containing the word "Chicken."

In [158]:
# Filter item names containing the word "Chicken"
chicken_items = chipotle_df.filter(instr(col("item_name"), "Chicken") > 0)

# Show the DataFrame with chicken items
chicken_items.show()
# chicken_items.show(truncate = False)

+---+--------+--------+--------------------+--------------------+----------+
|_c0|order_id|quantity|           item_name|  choice_description|item_price|
+---+--------+--------+--------------------+--------------------+----------+
|  4|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |
|  5|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $10.98 |
| 11|       6|       1|Chicken Crispy Tacos|[Roasted Chili Co...|    $8.75 |
| 12|       6|       1|  Chicken Soft Tacos|[Roasted Chili Co...|    $8.75 |
| 13|       7|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $11.25 |
| 16|       8|       1|     Chicken Burrito|[Tomatillo-Green ...|    $8.49 |
| 17|       9|       1|     Chicken Burrito|[Fresh Tomato Sal...|    $8.49 |
| 19|      10|       1|        Chicken Bowl|[Tomatillo Red Ch...|    $8.75 |
| 23|      12|       1|     Chicken Burrito|[[Tomatillo-Green...|   $10.98 |
| 26|      13|       1|        Chicken Bowl|[Roasted Chili Co...|    $8.49 |

23/09/01 16:51:43 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , order_id, quantity, item_name, choice_description, item_price
 Schema: _c0, order_id, quantity, item_name, choice_description, item_price
Expected: _c0 but found: 
CSV file: file:///home/riyaz/Documents/spark_training/Sparks-repo/day3/chipotle.csv


### Regular Expressions:

Question: Load the "chipotle" dataset and find the items with names that start with "Ch" followed by any character.



In [159]:
items_with_ch = chipotle_df.filter(col("item_name").like("Ch%"))
# items_with_ch.show()
items_with_ch.show(truncate=False)

+---+--------+--------+-------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------+----------+
|_c0|order_id|quantity|item_name                            |choice_description                                                                                                                    |item_price|
+---+--------+--------+-------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------+----------+
|0  |1       |1       |Chips and Fresh Tomato Salsa         |null                                                                                                                                  |$2.39     |
|3  |1       |1       |Chips and Tomatillo-Green Chili Salsa|null                                                                                                       

23/09/01 16:51:47 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , order_id, quantity, item_name, choice_description, item_price
 Schema: _c0, order_id, quantity, item_name, choice_description, item_price
Expected: _c0 but found: 
CSV file: file:///home/riyaz/Documents/spark_training/Sparks-repo/day3/chipotle.csv


### Working with Nulls in Data:

Question: Load the "titanic" dataset and count the number of passengers with missing age information.



In [160]:
missing_age_count = titanic_df.filter(col("Age").isNull()).count()
print("Number of passengers with missing age information:", missing_age_count)


Number of passengers with missing age information: 177


### Coalesce
Question: Utilizing the Chipotle dataset, use the coalesce function to combine the "item_name" and "choice_description" columns into a new column named "OrderDetails." Display the first 5 rows of the resulting DataFrame.

In [161]:
chipotle_df_coalesce = chipotle_df.withColumn("OrderDetails", coalesce(col("item_name"), col("choice_description")))

# Display the first 5 rows of the resulting DataFrame
# chipotle_df.show(5, truncate=False)
chipotle_df_coalesce.show(5)

+---+--------+--------+--------------------+--------------------+----------+--------------------+
|_c0|order_id|quantity|           item_name|  choice_description|item_price|        OrderDetails|
+---+--------+--------+--------------------+--------------------+----------+--------------------+
|  0|       1|       1|Chips and Fresh T...|                null|    $2.39 |Chips and Fresh T...|
|  1|       1|       1|                Izze|        [Clementine]|    $3.39 |                Izze|
|  2|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |    Nantucket Nectar|
|  3|       1|       1|Chips and Tomatil...|                null|    $2.39 |Chips and Tomatil...|
|  4|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |        Chicken Bowl|
+---+--------+--------+--------------------+--------------------+----------+--------------------+
only showing top 5 rows



23/09/01 16:51:52 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , order_id, quantity, item_name, choice_description, item_price
 Schema: _c0, order_id, quantity, item_name, choice_description, item_price
Expected: _c0 but found: 
CSV file: file:///home/riyaz/Documents/spark_training/Sparks-repo/day3/chipotle.csv


### ifnull, nullIf, nvl, and nvl2

Question: Replace the null values in the "Age" column of the Titanic dataset with the average age.

In [162]:
# Calculate average age excluding null values
average_age = titanic_df.select(avg(col("Age"))).collect()[0][0]

# Replace null values in the "Age" column with the average age
titanic_df1 = titanic_df.withColumn("Age", when(col("Age").isNull(), average_age).otherwise(col("Age")))

titanic_df1.show()

+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|              Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|             22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|             38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|             26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|             35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|             35.0|    0|    0|          373450|

### drop

Question: Remove the "Cabin" column from the Titanic dataset.


In [163]:
dropped = titanic_df.drop(col("Cabin"))
dropped.show()
# titanic_df.drop("Cabin").columns
# titanic_df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|          330877| 8.4583|       Q|
|          7|      

### fill

Question: Fill the null values in the "Age" column of the Titanic dataset with a default age of 30.

In [149]:
titanic_df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [164]:
default_age = 30

titanic_df2 = titanic_df.withColumn("Age", when(col("Age").isNull(), default_age).otherwise(col("Age"))).show()

# titanic_df2.show()


+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|30.0|    0|    0|      

###  replace

Question: Replace the gender "male" with "M" and "female" with "F" in the "Sex" column of the Titanic dataset.

In [165]:
# titanic_dfmf = titanic_df.withColumn("Sex", regexp_replace(col("Sex"), "Male", "M"))
# titanic_dfmf = titanic_df.withColumn("Sex", regexp_replace(col("Sex"), "Female", "F"))
# titanic_df.show()

titanic_df_mf = titanic_df.replace({"male":"m","female":"f"},"sex").show()


/home/riyaz/.local/lib/python3.10/site-packages/pyspark/sql/dataframe.py:4317: UserWarning: to_replace is a dict and value is not None. value will be ignored.
  warnings.warn("to_replace is a dict and value is not None. value will be ignored.")


+-----------+--------+------+--------------------+---+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+---+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  m|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|  f|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|  f|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|  f|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  m|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  m|null|    0|    0|          330877| 8.4583| null|  

### 6. Working with Complex Types: Structs

Question: Create a new DataFrame from the Kalimati Tarkari dataset, including a new column "PriceRange" that is a struct containing "Minimum" and "Maximum" prices for each commodity.

In [166]:
structKalimati_df = kalimati_df.selectExpr("*","struct(Minimum,Maximum) as PriceRange")
structKalimati_df.show()


+---+--------------------+----------+----+-------+-------+-------+------------+
| SN|           Commodity|      Date|Unit|Minimum|Maximum|Average|  PriceRange|
+---+--------------------+----------+----+-------+-------+-------+------------+
|  0|  Tomato Big(Nepali)|2013-06-16|  Kg|   35.0|   40.0|   37.5|{35.0, 40.0}|
|  1| Tomato Small(Local)|2013-06-16|  Kg|   26.0|   32.0|   29.0|{26.0, 32.0}|
|  2|          Potato Red|2013-06-16|  Kg|   20.0|   21.0|   20.5|{20.0, 21.0}|
|  3|        Potato White|2013-06-16|  Kg|   15.0|   16.0|   15.5|{15.0, 16.0}|
|  4|  Onion Dry (Indian)|2013-06-16|  Kg|   28.0|   30.0|   29.0|{28.0, 30.0}|
|  5|       Carrot(Local)|2013-06-16|  Kg|   30.0|   35.0|   32.5|{30.0, 35.0}|
|  6|      Cabbage(Local)|2013-06-16|  Kg|    6.0|   10.0|    8.0| {6.0, 10.0}|
|  7|         Cauli Local|2013-06-16|  Kg|   30.0|   35.0|   32.5|{30.0, 35.0}|
|  8|         Raddish Red|2013-06-16|  Kg|   35.0|   40.0|   37.5|{35.0, 40.0}|
|  9|Raddish White(Local)|2013-06-16|  K

### Working with Complex Types: Arrays
Question: Create a new DataFrame from the Kalimati Tarkari dataset, including a new column "CommodityList" that is an array of all the commodities.


In [167]:
Array_kalimati_df = kalimati_df.selectExpr("*","array(Commodity) as CommodityList")
Array_kalimati_df.show(truncate = False)

+---+--------------------+----------+----+-------+-------+-------+----------------------+
|SN |Commodity           |Date      |Unit|Minimum|Maximum|Average|CommodityList         |
+---+--------------------+----------+----+-------+-------+-------+----------------------+
|0  |Tomato Big(Nepali)  |2013-06-16|Kg  |35.0   |40.0   |37.5   |[Tomato Big(Nepali)]  |
|1  |Tomato Small(Local) |2013-06-16|Kg  |26.0   |32.0   |29.0   |[Tomato Small(Local)] |
|2  |Potato Red          |2013-06-16|Kg  |20.0   |21.0   |20.5   |[Potato Red]          |
|3  |Potato White        |2013-06-16|Kg  |15.0   |16.0   |15.5   |[Potato White]        |
|4  |Onion Dry (Indian)  |2013-06-16|Kg  |28.0   |30.0   |29.0   |[Onion Dry (Indian)]  |
|5  |Carrot(Local)       |2013-06-16|Kg  |30.0   |35.0   |32.5   |[Carrot(Local)]       |
|6  |Cabbage(Local)      |2013-06-16|Kg  |6.0    |10.0   |8.0    |[Cabbage(Local)]      |
|7  |Cauli Local         |2013-06-16|Kg  |30.0   |35.0   |32.5   |[Cauli Local]         |
|8  |Raddi

### Working with Complex Types: explode

Question: Explode the "CommodityList" array column from the previous step to generate a new row for each commodity in the list.

In [168]:
Array_kalimati_df.withColumn("exploded",explode("CommodityList")).show(5)


+---+-------------------+----------+----+-------+-------+-------+--------------------+-------------------+
| SN|          Commodity|      Date|Unit|Minimum|Maximum|Average|       CommodityList|           exploded|
+---+-------------------+----------+----+-------+-------+-------+--------------------+-------------------+
|  0| Tomato Big(Nepali)|2013-06-16|  Kg|   35.0|   40.0|   37.5|[Tomato Big(Nepali)]| Tomato Big(Nepali)|
|  1|Tomato Small(Local)|2013-06-16|  Kg|   26.0|   32.0|   29.0|[Tomato Small(Loc...|Tomato Small(Local)|
|  2|         Potato Red|2013-06-16|  Kg|   20.0|   21.0|   20.5|        [Potato Red]|         Potato Red|
|  3|       Potato White|2013-06-16|  Kg|   15.0|   16.0|   15.5|      [Potato White]|       Potato White|
|  4| Onion Dry (Indian)|2013-06-16|  Kg|   28.0|   30.0|   29.0|[Onion Dry (Indian)]| Onion Dry (Indian)|
+---+-------------------+----------+----+-------+-------+-------+--------------------+-------------------+
only showing top 5 rows



### Working with Complex Types: Maps

Question: Create a new DataFrame from the Kalimati Tarkari dataset, including a new column "PriceMap" that is a map with "Commodity" as the key and "Average" price as the value.
Answer:

In [169]:
map_kalimati = kalimati_df.select("*",create_map(col("Commodity"),col("Average")).alias("PriceMap"))
map_kalimati.show()

+---+--------------------+----------+----+-------+-------+-------+--------------------+
| SN|           Commodity|      Date|Unit|Minimum|Maximum|Average|            PriceMap|
+---+--------------------+----------+----+-------+-------+-------+--------------------+
|  0|  Tomato Big(Nepali)|2013-06-16|  Kg|   35.0|   40.0|   37.5|{Tomato Big(Nepal...|
|  1| Tomato Small(Local)|2013-06-16|  Kg|   26.0|   32.0|   29.0|{Tomato Small(Loc...|
|  2|          Potato Red|2013-06-16|  Kg|   20.0|   21.0|   20.5|{Potato Red -> 20.5}|
|  3|        Potato White|2013-06-16|  Kg|   15.0|   16.0|   15.5|{Potato White -> ...|
|  4|  Onion Dry (Indian)|2013-06-16|  Kg|   28.0|   30.0|   29.0|{Onion Dry (India...|
|  5|       Carrot(Local)|2013-06-16|  Kg|   30.0|   35.0|   32.5|{Carrot(Local) ->...|
|  6|      Cabbage(Local)|2013-06-16|  Kg|    6.0|   10.0|    8.0|{Cabbage(Local) -...|
|  7|         Cauli Local|2013-06-16|  Kg|   30.0|   35.0|   32.5|{Cauli Local -> 3...|
|  8|         Raddish Red|2013-0

### Working with JSON

Question: Convert the "kalimati_df" DataFrame to JSON format and write it to a JSON file.

In [170]:
from pyspark.sql.functions import to_json,from_json

jsonKalimati = kalimati_df.selectExpr("(Sn,Commodity,date,unit,minimum,maximum,average) as KalimatiJson").select(to_json(col("KalimatiJson")))
jsonKalimati.show()
# jsonKalimati.write.json("kalimatiDFJson.json")


+---------------------+
|to_json(KalimatiJson)|
+---------------------+
| {"Sn":0,"Commodit...|
| {"Sn":1,"Commodit...|
| {"Sn":2,"Commodit...|
| {"Sn":3,"Commodit...|
| {"Sn":4,"Commodit...|
| {"Sn":5,"Commodit...|
| {"Sn":6,"Commodit...|
| {"Sn":7,"Commodit...|
| {"Sn":8,"Commodit...|
| {"Sn":9,"Commodit...|
| {"Sn":10,"Commodi...|
| {"Sn":11,"Commodi...|
| {"Sn":12,"Commodi...|
| {"Sn":13,"Commodi...|
| {"Sn":14,"Commodi...|
| {"Sn":15,"Commodi...|
| {"Sn":16,"Commodi...|
| {"Sn":17,"Commodi...|
| {"Sn":18,"Commodi...|
| {"Sn":19,"Commodi...|
+---------------------+
only showing top 20 rows

